### Cellule 1 : Importation des librairies essentielles python

In [50]:
# Import des librairies essentielles
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


### Cellule 2 : Importation des données

In [51]:
# Définir le chemin relatif
file_path = os.path.join("..", "data", "flipkart_com-ecommerce_sample_1050.csv")

# Chargement du dataset des produits
data = pd.read_csv(file_path)
print("Dimensions du dataset :", data.shape)


Dimensions du dataset : (1050, 15)


### Cellule 3 : Affichage d'un aperçu des données

In [52]:
# Afficher les premières lignes du dataset
data.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,55b85ea15a1536d46b7190ad6fff8ce7,2016-04-30 03:22:56 +0000,http://www.flipkart.com/elegance-polyester-mul...,Elegance Polyester Multicolor Abstract Eyelet ...,"[""Home Furnishing >> Curtains & Accessories >>...",CRNEG7BKMFFYHQ8Z,1899.0,899.0,55b85ea15a1536d46b7190ad6fff8ce7.jpg,False,Key Features of Elegance Polyester Multicolor ...,No rating available,No rating available,Elegance,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
1,7b72c92c2f6c40268628ec5f14c6d590,2016-04-30 03:22:56 +0000,http://www.flipkart.com/sathiyas-cotton-bath-t...,Sathiyas Cotton Bath Towel,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEGFZHGBXPHZUH,600.0,449.0,7b72c92c2f6c40268628ec5f14c6d590.jpg,False,Specifications of Sathiyas Cotton Bath Towel (...,No rating available,No rating available,Sathiyas,"{""product_specification""=>[{""key""=>""Machine Wa..."
2,64d5d4a258243731dc7bbb1eef49ad74,2016-04-30 03:22:56 +0000,http://www.flipkart.com/eurospa-cotton-terry-f...,Eurospa Cotton Terry Face Towel Set,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEG6SHXTDB2A2Y,NaN,NaN,64d5d4a258243731dc7bbb1eef49ad74.jpg,False,Key Features of Eurospa Cotton Terry Face Towe...,No rating available,No rating available,Eurospa,"{""product_specification""=>[{""key""=>""Material"",..."
3,d4684dcdc759dd9cdf41504698d737d8,2016-06-20 08:49:52 +0000,http://www.flipkart.com/santosh-royal-fashion-...,SANTOSH ROYAL FASHION Cotton Printed King size...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",BDSEJT9UQWHDUBH4,2699.0,1299.0,d4684dcdc759dd9cdf41504698d737d8.jpg,False,Key Features of SANTOSH ROYAL FASHION Cotton P...,No rating available,No rating available,SANTOSH ROYAL FASHION,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
4,6325b6870c54cd47be6ebfbffa620ec7,2016-06-20 08:49:52 +0000,http://www.flipkart.com/jaipur-print-cotton-fl...,Jaipur Print Cotton Floral King sized Double B...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",BDSEJTHNGWVGWWQU,2599.0,698.0,6325b6870c54cd47be6ebfbffa620ec7.jpg,False,Key Features of Jaipur Print Cotton Floral Kin...,No rating available,No rating available,Jaipur Print,"{""product_specification""=>[{""key""=>""Machine Wa..."


### Cellule 4 : Vérification des colonnes disponibles

In [53]:
# Afficher les noms des colonnes
print("Colonnes du dataset :", data.columns.tolist())


Colonnes du dataset : ['uniq_id', 'crawl_timestamp', 'product_url', 'product_name', 'product_category_tree', 'pid', 'retail_price', 'discounted_price', 'image', 'is_FK_Advantage_product', 'description', 'product_rating', 'overall_rating', 'brand', 'product_specifications']


### Cellule 9 : Préparation des catégories

In [54]:
import os
import pandas as pd

# Extraire la première catégorie de la colonne 'product_category_tree'
data['category'] = data['product_category_tree'].apply(lambda x: x.split('>>')[0].strip())

# Mettre les catégories en minuscules pour uniformiser
data['category'] = data['category'].apply(lambda x: x.lower())

# Remplacer les valeurs manquantes dans 'category' par 'unknown'
data['category'] = data['category'].fillna('unknown')

# Liste des catégories uniques
categories = data['category'].unique().tolist()
print("Catégories :", categories)

# Mapping des catégories vers des numéros
category_to_num = {category: idx for idx, category in enumerate(categories)}
data['category_num'] = data['category'].map(category_to_num)

# Extraire les numéros de catégories
y_cat_num = data['category_num'].values
print("Numéros de catégories :", y_cat_num)


Catégories : ['["home furnishing', '["baby care', '["watches', '["home decor & festive needs', '["kitchen & dining', '["beauty and personal care', '["computers']
Numéros de catégories : [0 1 1 ... 1 1 1]


### Cellule 5 : Préparation du traitement du texte

In [55]:
# Import des librairies pour le traitement du texte
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Téléchargement des ressources NLTK (à exécuter une seule fois)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pat\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [56]:
import nltk
from nltk.tokenize import word_tokenize

# Exemple de texte
texte = "Bonjour, ceci est un test de tokenisation."

# Tokenisation du texte
tokens = word_tokenize(texte)

# Affichage des tokens
print(tokens)

['Bonjour', ',', 'ceci', 'est', 'un', 'test', 'de', 'tokenisation', '.']


### Cellule 6 : Définition des fonctions de prétraitement

In [57]:
# Fonction de tokenisation
def tokenizer_fct(sentence):
    sentence_clean = sentence.replace('-', ' ').replace('+', ' ').replace('/', ' ').replace('#', ' ')
    word_tokens = word_tokenize(sentence_clean)
    return word_tokens

# Liste des stop words
stop_w = set(stopwords.words('english'))
punctuations = ['[', ']', ',', '.', ':', '?', '(', ')', '!', ';', '"', "'", '&', '%', '$', '@', '#', '*', '–', '—', '...']
stop_w.update(punctuations)

# Fonction de filtrage des stop words
def stop_word_filter_fct(list_words):
    filtered_w = [w for w in list_words if not w.lower() in stop_w]
    filtered_w2 = [w for w in filtered_w if len(w) > 2]
    return filtered_w2

# Fonction de mise en minuscule et suppression de certains mots
def lower_start_fct(list_words):
    lw = [w.lower() for w in list_words if not w.startswith("@") and not w.startswith("#") and not w.startswith("http")]
    return lw

# Fonction de lemmatisation
def lemma_fct(list_words):
    lemmatizer = WordNetLemmatizer()
    lem_w = [lemmatizer.lemmatize(w) for w in list_words]
    return lem_w

# Fonction de préparation du texte pour Bag-of-Words
def transform_bow_fct(desc_text):
    word_tokens = tokenizer_fct(desc_text)
    sw = stop_word_filter_fct(word_tokens)
    lw = lower_start_fct(sw)
    transf_desc_text = ' '.join(lw)
    return transf_desc_text

# Fonction de préparation du texte pour Bag-of-Words avec lemmatisation
def transform_bow_lem_fct(desc_text):
    word_tokens = tokenizer_fct(desc_text)
    sw = stop_word_filter_fct(word_tokens)
    lw = lower_start_fct(sw)
    lem_w = lemma_fct(lw)
    transf_desc_text = ' '.join(lem_w)
    return transf_desc_text

# Fonction de préparation du texte pour les modèles de Deep Learning (BERT, USE)
def transform_dl_fct(desc_text):
    word_tokens = tokenizer_fct(desc_text)
    lw = lower_start_fct(word_tokens)
    transf_desc_text = ' '.join(lw)
    return transf_desc_text


### Cellule 7 : Application du prétraitement

In [58]:
# Remplacer les valeurs manquantes dans 'description' par une chaîne vide
data['description'] = data['description'].fillna('')

# Application des fonctions de prétraitement
data['sentence_bow'] = data['description'].apply(lambda x: transform_bow_fct(str(x)))
data['sentence_bow_lem'] = data['description'].apply(lambda x: transform_bow_lem_fct(str(x)))
data['sentence_dl'] = data['description'].apply(lambda x: transform_dl_fct(str(x)))

print("Dimensions du dataset après prétraitement :", data.shape)


Dimensions du dataset après prétraitement : (1050, 20)


### Cellule 8 : Sauvegarde des données prétraitées (optionnel)

In [59]:
import os

# Définir le chemin relatif vers le dossier "data"
file_path = os.path.join("..", "data", "preprocessed_product_data.csv")

# Sauvegarde du dataset prétraité pour une utilisation future
data.to_csv(file_path, index=False)


In [60]:
print(data.columns)


Index(['uniq_id', 'crawl_timestamp', 'product_url', 'product_name',
       'product_category_tree', 'pid', 'retail_price', 'discounted_price',
       'image', 'is_FK_Advantage_product', 'description', 'product_rating',
       'overall_rating', 'brand', 'product_specifications', 'category',
       'category_num', 'sentence_bow', 'sentence_bow_lem', 'sentence_dl'],
      dtype='object')


### Cellule 10 : Extraction des features avec Bag-of-Words et TF-IDF


In [62]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Téléchargement des ressources NLTK (à exécuter une seule fois)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialiser les outils de NLTK
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Fonction pour nettoyer et lemmatiser les catégories de produits
def preprocess_text(text):
    text = text.replace('[', '').replace(']', '')  # Supprimer les crochets
    tokens = word_tokenize(text.lower())  # Conversion en minuscules et tokenisation
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]  # Lemmatisation et suppression des stop words
    return ' '.join(tokens)

# Appliquer la fonction de prétraitement à la colonne 'category'
data['sentence_bow_lem'] = data['category'].apply(preprocess_text)

# Vérifier la colonne nouvellement créée
print(data[['category', 'sentence_bow_lem']].head())


            category sentence_bow_lem
0  ["home furnishing  home furnishing
1        ["baby care        baby care
2        ["baby care        baby care
3  ["home furnishing  home furnishing
4  ["home furnishing  home furnishing


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pat\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [63]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Initialisation des vectoriseurs
cvect = CountVectorizer(stop_words='english', max_df=0.95, min_df=1)
tfidf_vect = TfidfVectorizer(stop_words='english', max_df=0.95, min_df=1)

# Sélection de la colonne à utiliser
feat = 'sentence_bow_lem'

# Ajustement et transformation des données
cv_fit = cvect.fit(data[feat])
tfidf_fit = tfidf_vect.fit(data[feat])

cv_transform = cvect.transform(data[feat])
tfidf_transform = tfidf_vect.transform(data[feat])


### Cellule 11 : Sauvegarde des embeddings Bag-of-Words

In [64]:
# Conversion en format dense (attention à la taille de la matrice)
cv_features = cv_transform.toarray()

# Vérification ou création de la colonne 'product_id'
if 'uniq_id' in data.columns:
    data.rename(columns={'uniq_id': 'product_id'}, inplace=True)
elif 'product_id' not in data.columns:
    data['product_id'] = data.index

product_ids = data['product_id'].values

# Sauvegarde des embeddings et des identifiants de produits
np.savez_compressed('countvectorizer_embeddings.npz', product_id=product_ids, embeddings=cv_features)


### Cellule 12 : Sauvegarde des embeddings TF-IDF

In [65]:
# Conversion en format dense
tfidf_features = tfidf_transform.toarray()

# Sauvegarde des embeddings et des identifiants de produits
np.savez_compressed('tfidf_embeddings.npz', product_id=product_ids, embeddings=tfidf_features)


### Cellule 13 : Extraction des features avec Word2Vec

In [67]:
import gensim

# Préparation des phrases pour Word2Vec
sentences = data['sentence_bow_lem'].apply(lambda x: x.split()).tolist()

# Paramètres du modèle Word2Vec
w2v_size = 300
w2v_window = 5
w2v_min_count = 1
w2v_epochs = 100

# Entraînement du modèle Word2Vec
print("Entraînement du modèle Word2Vec...")
w2v_model = gensim.models.Word2Vec(
    sentences,
    vector_size=w2v_size,
    window=w2v_window,
    min_count=w2v_min_count,
    workers=4,
    seed=42,
    epochs=w2v_epochs
)

# Fonction pour obtenir l'embedding d'un document
def document_vector(doc):
    doc = [word for word in doc if word in w2v_model.wv.key_to_index]
    return np.mean(w2v_model.wv[doc], axis=0) if len(doc) > 0 else np.zeros(w2v_size)

# Création des embeddings pour chaque description
w2v_embeddings = np.array([document_vector(doc) for doc in sentences])

# Sauvegarde des embeddings Word2Vec
np.savez_compressed('word2vec_embeddings.npz', product_id=product_ids, embeddings=w2v_embeddings)


Entraînement du modèle Word2Vec...


### Cellule 14 : Extraction des features avec BERT


In [73]:
from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf

# Paramètres
max_length = 64
batch_size = 32
model_name = 'bert-base-uncased'

# Chargement du tokenizer et du modèle BERT
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name)

# Préparation des inputs
sentences = data['sentence_dl'].tolist()

def bert_encode(sentences, tokenizer, max_length):
    input_ids = []
    attention_masks = []

    for sentence in sentences:
        encoded = tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='tf',
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return tf.concat(input_ids, axis=0), tf.concat(attention_masks, axis=0)

input_ids, attention_masks = bert_encode(sentences, tokenizer, max_length)

# Génération des embeddings
embeddings = []
num_examples = len(sentences)

for i in range(0, num_examples, batch_size):
    batch_input_ids = input_ids[i:i+batch_size]
    batch_attention_masks = attention_masks[i:i+batch_size]
    outputs = model(batch_input_ids, attention_mask=batch_attention_masks)
    # Utilisation de la représentation du token [CLS]
    batch_embeddings = outputs.last_hidden_state[:, 0, :].numpy()
    embeddings.append(batch_embeddings)

bert_embeddings = np.vstack(embeddings)

# Sauvegarde des embeddings BERT
np.savez_compressed('bert_embeddings.npz', product_id=product_ids, embeddings=bert_embeddings)


C:\Users\pat\.conda\envs\P6q\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


RuntimeError: Failed to import transformers.models.bert.modeling_tf_bert because of the following error (look up to see its traceback):
No module named 'keras.engine'

### Cellule 15 : Extraction des features avec Universal Sentence Encoder (USE)

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf

# Chargement du modèle USE
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Génération des embeddings
sentences = data['sentence_dl'].tolist()
batch_size = 32

embeddings = []
num_examples = len(sentences)

for i in range(0, num_examples, batch_size):
    batch_sentences = sentences[i:i+batch_size]
    batch_embeddings = embed(batch_sentences).numpy()
    embeddings.append(batch_embeddings)

use_embeddings = np.vstack(embeddings)

# Sauvegarde des embeddings USE
np.savez_compressed('use_embeddings.npz', product_id=product_ids, embeddings=use_embeddings)


### Cellule 16 : Sauvegarde des identifiants de produits et des catégories

In [ ]:
# Sauvegarde des identifiants de produits et des catégories
product_info = data[['product_id', 'product_category', 'category_num']]
product_info.to_csv('product_info.csv', index=False)


### Cellule 17 : Vérification des embeddings générés (optionnel)

In [ ]:
# Vérifier les dimensions des embeddings
print("Dimensions des embeddings CountVectorizer :", cv_features.shape)
print("Dimensions des embeddings TF-IDF :", tfidf_features.shape)
print("Dimensions des embeddings Word2Vec :", w2v_embeddings.shape)
print("Dimensions des embeddings BERT :", bert_embeddings.shape)
print("Dimensions des embeddings USE :", use_embeddings.shape)
